In [ ]:
# ==================================================
# 🔁 Full Reproducibility + Warning Suppression Setup
# ==================================================
import os
import random
import logging
import warnings

# === ENVIRONMENT VARIABLES (SET BEFORE TF IMPORT) ===
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)           # Hash seed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'           # Suppress TensorFlow logs
os.environ['TF_DETERMINISTIC_OPS'] = '0'           # Allow non-deterministic ops to avoid UnimplementedError
os.environ['CUDA_VISIBLE_DEVICES'] = '0'           # Set GPU ID (or "" to force CPU)

# === PYTHON SEED SETTINGS ===
random.seed(SEED)

# === SUPPRESS WARNINGS & LOGGING ===
logging.getLogger('absl').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# === IMPORT LIBRARIES AFTER SEED SETTINGS ===
import numpy as np
import tensorflow as tf
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from tqdm import tqdm

# === NUMPY & TENSORFLOW SEEDS ===
np.random.seed(SEED)
tf.random.set_seed(SEED)

# === SINGLE-THREADING FOR FULL REPRODUCIBILITY ===
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# ✅ CHECK GPU AVAILABILITY
print("✅ GPU Available:", tf.config.list_physical_devices('GPU'))

# ✅ DATASET PATH
DATASET_PATH = r'G:\498R\BanglaSER'

# ✅ EMOTION LABELS FROM FILENAME
# Format: Mode-StatementType-Emotion-Intensity-Statement-Repetition-Actor.wav
EMOTION_MAPPING = {
    '01': 'happy',
    '02': 'sad',
    '03': 'angry',
    '04': 'surprise',
    '05': 'neutral'
}

<h1>Hyperparameter Tuning<h1/>

In [ ]:
import os
import numpy as np
import tensorflow as tf

# ===============================
# 🔧 Configuration
# ===============================
SAVE_PATH = r"D:\498R"   # Adjust path
batch_size = 32
# ===============================
# 📂 Load datasets from disk
# ===============================
def load_dataset(filename):
    path = os.path.join(SAVE_PATH, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    data = np.load(path)
    X, y = data['X'], data['y']
    # Ensure correct dtype for TensorFlow
    X = X.astype('float32')
    y = y.astype('int64')
    return X, y

X_train, y_train = load_dataset("train(1000)(Cleaned).npz")
X_val, y_val     = load_dataset("val(1000)(Cleaned).npz")
X_test, y_test   = load_dataset("test(1000)(Cleaned).npz")

# ===============================
# 🔁 Create tf.data pipelines
# ===============================
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(buffer_size=len(X_train), seed=42) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)) \
    .batch(batch_size) \
    .cache() \
    .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
    .batch(batch_size) \
    .cache() \
    .prefetch(tf.data.AUTOTUNE)

# ===============================
# 🛠 Sanity checks
# ===============================
print(f"Train: {X_train.shape} {y_train.shape}")
print(f"Val:   {X_val.shape} {y_val.shape}")
print(f"Test:  {X_test.shape} {y_test.shape}")

print("Train classes:", np.unique(y_train))
print("Val classes:  ", np.unique(y_val))
print("Test classes: ", np.unique(y_test))

In [ ]:
import os, time, math, random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd

# -----------------------------
# Config
# -----------------------------
EPOCHS = 20                     # you set this to 20 in your latest code
SAVE_PATH = r"D:\498R"
SEED = 42
os.makedirs(SAVE_PATH, exist_ok=True)

# Repro
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -----------------------------
# Optional: load datasets if not already present in memory
# -----------------------------
def _load_npz_dataset(base_path, fname):
    path = os.path.join(base_path, fname)
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    d = np.load(path)
    X = d['X'].astype('float32')
    y = d['y'].astype('int64')
    return X, y

if 'X_train' not in globals():
    X_train, y_train = _load_npz_dataset(SAVE_PATH, "train(1000)(Cleaned).npz")
    X_val,   y_val   = _load_npz_dataset(SAVE_PATH, "val(1000)(Cleaned).npz")
    X_test,  y_test  = _load_npz_dataset(SAVE_PATH, "test(1000)(Cleaned).npz")

print(f"Train: {X_train.shape} {y_train.shape}")
print(f"Val:   {X_val.shape} {y_val.shape}")
print(f"Test:  {X_test.shape} {y_test.shape}")

# -----------------------------
# Shapes & labels
# -----------------------------
input_shape = X_train.shape[1:]         # e.g., (126, 155)
num_labels  = int(np.max([y_train.max(), y_val.max(), y_test.max()])) + 1

# -----------------------------
# Model (logits out; use from_logits=True)
# -----------------------------
def create_cnn_model(input_shape, num_labels):
    inputs = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv1D(128, 5, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Dropout(0.3, seed=SEED)(x)

    x = tf.keras.layers.Conv1D(256, 5, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Dropout(0.3, seed=SEED)(x)

    x = tf.keras.layers.Conv1D(512, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Dropout(0.3, seed=SEED)(x)

    x = tf.keras.layers.Conv1D(1024, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Dropout(0.3, seed=SEED)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.4, seed=SEED)(x)

    # logits (no softmax)
    outputs = tf.keras.layers.Dense(num_labels)(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# -----------------------------
# Dataset builder per batch size
# -----------------------------
def make_tf_datasets(Xtr, ytr, Xva, yva, Xte, yte, batch_size):
    train_ds = tf.data.Dataset.from_tensor_slices((Xtr, ytr)) \
        .shuffle(len(Xtr), seed=SEED) \
        .batch(batch_size) \
        .prefetch(tf.data.AUTOTUNE)
    val_ds   = tf.data.Dataset.from_tensor_slices((Xva, yva)) \
        .batch(batch_size) \
        .cache() \
        .prefetch(tf.data.AUTOTUNE)
    test_ds  = tf.data.Dataset.from_tensor_slices((Xte, yte)) \
        .batch(batch_size) \
        .cache() \
        .prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds, test_ds

# -----------------------------
# Optimizer / Scheduler helpers
# -----------------------------
def get_optimizer(name, lr):
    name = name.lower()
    if name == 'adam':
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if name == 'rmsprop':
        return tf.keras.optimizers.RMSprop(learning_rate=lr)
    if name == 'sgd':
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    raise ValueError(f"Unknown optimizer: {name}")

def get_scheduler_and_lr(sched_name, base_lr, steps_per_epoch, epochs):
    """
    Returns (learning_rate_for_optimizer, callbacks_list)
    - 'cosine': tf.keras CosineDecay schedule (no callback)
    - 'step': LearningRateScheduler (half at halfway)
    - 'plateau': ReduceLROnPlateau on val_loss
    """
    sched_name = sched_name.lower()
    if sched_name == 'cosine':
        decay_steps = max(1, steps_per_epoch * epochs)
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=base_lr, decay_steps=decay_steps
        )
        return lr_schedule, []
    elif sched_name == 'step':
        def step_fn(epoch, lr):
            return base_lr * 0.5 if epoch >= (epochs // 2) else base_lr
        cb = tf.keras.callbacks.LearningRateScheduler(step_fn, verbose=0)
        return base_lr, [cb]
    elif sched_name == 'plateau':
        cb = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=0
        )
        return base_lr, [cb]
    else:
        raise ValueError(f"Unknown scheduler: {sched_name}")

# -----------------------------
# Evaluation on val set
# -----------------------------
def eval_on_val(model, val_ds):
    y_true, y_pred = [], []
    for xb, yb in val_ds:
        logits = model.predict(xb, verbose=0)
        y_true.append(yb.numpy())
        y_pred.append(np.argmax(logits, axis=1))
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    return f1, acc

# -----------------------------
# Hyperparameter grid
# -----------------------------
OPTIMIZERS = ['Adam', 'RMSprop', 'SGD']
BATCH_SIZES = [16, 32, 64]
SCHEDULERS = ['Cosine', 'Step', 'Plateau']
LRS = [1e-2, 2e-3, 1e-4, 4e-4, 1e-3]   # keep this order

# -----------------------------
# Run grid search
# -----------------------------
results = []
total_runs = len(OPTIMIZERS)*len(BATCH_SIZES)*len(SCHEDULERS)*len(LRS)
run_idx = 0

for opt_name in OPTIMIZERS:
    for bs in BATCH_SIZES:
        train_ds, val_ds, test_ds = make_tf_datasets(X_train, y_train, X_val, y_val, X_test, y_test, bs)
        steps_per_epoch = math.ceil(len(X_train)/bs)
        for sched_name in SCHEDULERS:
            for lr in LRS:
                run_idx += 1
                tf.keras.backend.clear_session()
                model = create_cnn_model(input_shape, num_labels)

                # compile with optimizer + scheduler; logits => from_logits=True
                lr_or_schedule, sched_cbs = get_scheduler_and_lr(sched_name, lr, steps_per_epoch, EPOCHS)
                optimizer = get_optimizer(opt_name, lr_or_schedule)
                loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                model.compile(optimizer=optimizer,
                              loss=loss_obj,
                              metrics=['accuracy'])

                # train (fixed epochs; no early stopping to keep runs comparable)
                start = time.time()
                history = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    epochs=EPOCHS,
                    verbose=0,
                    callbacks=sched_cbs
                )
                train_time = time.time() - start

                # evaluate on val (macro-F1 + accuracy)
                val_f1, val_acc = eval_on_val(model, val_ds)

                results.append({
                    "optimizer": opt_name,
                    "batch_size": bs,
                    "scheduler": sched_name,
                    "lr": lr,
                    "val_macro_f1": val_f1,
                    "val_accuracy": val_acc,
                    "train_time_sec": train_time,
                    "final_val_loss": float(history.history['val_loss'][-1]),
                    "final_val_acc":  float(history.history['val_accuracy'][-1]),
                })
                print(f"[{run_idx:3d}/{total_runs}] opt={opt_name:7s} bs={bs:2d} "
                      f"sch={sched_name:7s} lr={lr:.0e}  F1={val_f1:.4f}  Acc={val_acc:.4f}")

# -----------------------------
# Results as DataFrame + CSV
# -----------------------------
df = pd.DataFrame(results).sort_values(by=['val_macro_f1','val_accuracy'], ascending=False)
csv_path = os.path.join(SAVE_PATH, "gridsearch_results.csv")
df.to_csv(csv_path, index=False)
print(f"\nSaved results to: {csv_path}")
print(df.head(10))

# -----------------------------
# Plots
# -----------------------------
def grouped_bar(ax, labels, vals1, vals2, legend1, legend2, title, ylabel):
    x = np.arange(len(labels))
    w = 0.35
    ax.bar(x - w/2, vals1, width=w, label=legend1)
    ax.bar(x + w/2, vals2, width=w, label=legend2)
    ax.set_xticks(x, labels)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.3)

# 1) Optimizer bar chart (aggregate across other params: mean)
agg_opt = df.groupby('optimizer')[['val_macro_f1','val_accuracy']].mean().reindex(OPTIMIZERS)
fig, ax = plt.subplots(figsize=(7,5))
grouped_bar(ax,
            labels=agg_opt.index.tolist(),
            vals1=agg_opt['val_macro_f1'].values,
            vals2=agg_opt['val_accuracy'].values,
            legend1='Val Macro-F1', legend2='Val Acc',
            title='Optimizer Comparison (mean across grid)',
            ylabel='Score')
plt.tight_layout()
opt_png = os.path.join(SAVE_PATH, "hp_optimizer_bar.png")
plt.savefig(opt_png, dpi=120); plt.close()
print(f"Saved: {opt_png}")

# 2) Scheduler bar chart (aggregate across other params: mean)
agg_sch = df.groupby('scheduler')[['val_macro_f1','val_accuracy']].mean().reindex(SCHEDULERS)
fig, ax = plt.subplots(figsize=(7,5))
grouped_bar(ax,
            labels=agg_sch.index.tolist(),
            vals1=agg_sch['val_macro_f1'].values,
            vals2=agg_sch['val_accuracy'].values,
            legend1='Val Macro-F1', legend2='Val Acc',
            title='LR Scheduler Comparison (mean across grid)',
            ylabel='Score')
plt.tight_layout()
sch_png = os.path.join(SAVE_PATH, "hp_scheduler_bar.png")
plt.savefig(sch_png, dpi=120); plt.close()
print(f"Saved: {sch_png}")

# 3) LR sensitivity (aggregate: mean over other params)
agg_lr = df.groupby('lr')[['val_macro_f1','val_accuracy']].mean().reindex(LRS)
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(agg_lr.index.values, agg_lr['val_macro_f1'].values, marker='o', label='Val Macro-F1')
ax.plot(agg_lr.index.values, agg_lr['val_accuracy'].values, marker='o', label='Val Acc')
ax.set_xscale('log')
ax.set_xlabel('Learning Rate (log scale)')
ax.set_ylabel('Score')
ax.set_title('Learning Rate Sensitivity (mean across grid)')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
lr_png = os.path.join(SAVE_PATH, "hp_lr_sensitivity.png")
plt.savefig(lr_png, dpi=120); plt.close()
print(f"Saved: {lr_png}")

# -----------------------------
# Print the best config
# -----------------------------
best = df.iloc[0]
print("\nBest config by Val Macro-F1:")
print(best)

In [ ]:
# -----------------------------
# Clean, padded plots (white bg, no grid, pastel palettes)
# -----------------------------
import matplotlib.pyplot as plt

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

# pastel palettes (two colors per grouped bar fig, one pair per figure)
PALETTE_OPT   = ('#D7A4A9', '#A86380')  # rose + plum
PALETTE_SCHED = ('#9CC8B0', '#5E9C8B')  # mint + teal
PALETTE_BS    = ('#BBB3E3', '#7C82B6')  # lavender + slate
LINE_F1_COLOR  = '#E67E22'              # orange
LINE_ACC_COLOR = '#2E8B57'              # sea green

def _add_headroom(ax, top_pad=0.08, bottom_pad=0.02):
    """Give axes some headroom so labels don't touch borders."""
    ymin, ymax = ax.get_ylim()
    rng = ymax - ymin if ymax > ymin else 1.0
    ax.set_ylim(ymin - bottom_pad*rng, ymax + top_pad*rng)

def _format_val(v):
    # If scores look like 0–1, print as %; else keep 3 decimals.
    return f"{v*100:.1f}%" if 0.0 <= v <= 1.5 else f"{v:.3f}"

def add_bar_labels(ax, bars, fontsize=18, y_offset_frac=0.015):
    ymin, ymax = ax.get_ylim()
    rng = ymax - ymin if ymax > ymin else 1.0
    for b in bars:
        h = b.get_height()
        ax.text(b.get_x() + b.get_width()/2,
                h + y_offset_frac*rng,
                _format_val(h),
                ha='center', va='bottom', fontsize=fontsize, clip_on=False)

def grouped_bar(ax, labels, vals1, vals2, legend1, legend2, title, ylabel,
                colors=('C0','C1')):
    x = np.arange(len(labels))
    w = 0.36

    b1 = ax.bar(x - w/2, vals1, width=w, label=legend1, color=colors[0])
    b2 = ax.bar(x + w/2, vals2, width=w, label=legend2, color=colors[1])

    ax.set_xticks(x, labels)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()
    ax.set_facecolor('white')  # no grid / white bg
    # padding + labels
    _add_headroom(ax, top_pad=0.10)
    add_bar_labels(ax, b1, fontsize=18)
    add_bar_labels(ax, b2, fontsize=18)

# 1) Optimizer bar chart (aggregate across other params: mean)
agg_opt = df.groupby('optimizer')[['val_macro_f1','val_accuracy']].mean().reindex(OPTIMIZERS)
fig, ax = plt.subplots(figsize=(8,6))
grouped_bar(ax,
            labels=agg_opt.index.tolist(),
            vals1=agg_opt['val_macro_f1'].values,
            vals2=agg_opt['val_accuracy'].values,
            legend1='Val Macro-F1', legend2='Val Acc',
            title='Optimizer Comparison (mean across grid)',
            ylabel='Score',
            colors=PALETTE_OPT)
plt.tight_layout()
opt_png = os.path.join(SAVE_PATH, "hp_optimizer_bar.png")
plt.savefig(opt_png, dpi=150, bbox_inches='tight', pad_inches=0.25); plt.close()
print(f"Saved: {opt_png}")

# 2) Scheduler bar chart (aggregate across other params: mean)
agg_sch = df.groupby('scheduler')[['val_macro_f1','val_accuracy']].mean().reindex(SCHEDULERS)
fig, ax = plt.subplots(figsize=(8,6))
grouped_bar(ax,
            labels=agg_sch.index.tolist(),
            vals1=agg_sch['val_macro_f1'].values,
            vals2=agg_sch['val_accuracy'].values,
            legend1='Val Macro-F1', legend2='Val Acc',
            title='LR Scheduler Comparison (mean across grid)',
            ylabel='Score',
            colors=PALETTE_SCHED)
plt.tight_layout()
sch_png = os.path.join(SAVE_PATH, "hp_scheduler_bar.png")
plt.savefig(sch_png, dpi=150, bbox_inches='tight', pad_inches=0.25); plt.close()
print(f"Saved: {sch_png}")

# 3) Batch size bar chart (aggregate across other params: mean)
agg_bs = df.groupby('batch_size')[['val_macro_f1','val_accuracy']].mean().reindex(BATCH_SIZES)
fig, ax = plt.subplots(figsize=(8,6))
grouped_bar(ax,
            labels=[str(x) for x in agg_bs.index.tolist()],
            vals1=agg_bs['val_macro_f1'].values,
            vals2=agg_bs['val_accuracy'].values,
            legend1='Val Macro-F1', legend2='Val Acc',
            title='Batch Size Comparison (mean across grid)',
            ylabel='Score',
            colors=PALETTE_BS)
plt.tight_layout()
bs_png = os.path.join(SAVE_PATH, "hp_batchsize_bar.png")
plt.savefig(bs_png, dpi=150, bbox_inches='tight', pad_inches=0.25); plt.close()
print(f"Saved: {bs_png}")

# 4) LR sensitivity (aggregate: mean over other params) with padded labels
agg_lr = df.groupby('lr')[['val_macro_f1','val_accuracy']].mean().reindex(LRS)
fig, ax = plt.subplots(figsize=(8,6))

x_lr = np.array(agg_lr.index.values, dtype=float)
y_f1 = agg_lr['val_macro_f1'].values
y_acc = agg_lr['val_accuracy'].values

l1, = ax.plot(x_lr, y_f1, marker='o', linewidth=2.5, markersize=8,
              label='Val Macro-F1', color=LINE_F1_COLOR)
l2, = ax.plot(x_lr, y_acc, marker='o', linewidth=2.5, markersize=8,
              label='Val Acc', color=LINE_ACC_COLOR)

# add a little headroom and per-point labels
_add_headroom(ax, top_pad=0.12)
ymin, ymax = ax.get_ylim(); rng = ymax - ymin if ymax > ymin else 1.0
for xv, yv in zip(x_lr, y_f1):
    ax.text(xv, yv + 0.015*rng, _format_val(yv), ha='center', va='bottom', fontsize=18, clip_on=False)
for xv, yv in zip(x_lr, y_acc):
    ax.text(xv, yv + 0.015*rng, _format_val(yv), ha='center', va='bottom', fontsize=18, clip_on=False)

ax.set_xscale('log')
ax.set_xlabel('Learning Rate (log scale)')
ax.set_ylabel('Score')
ax.set_title('Learning Rate Sensitivity (mean across grid)')
ax.legend()
ax.set_facecolor('white')
plt.tight_layout()
lr_png = os.path.join(SAVE_PATH, "hp_lr_sensitivity.png")
plt.savefig(lr_png, dpi=150, bbox_inches='tight', pad_inches=0.25); plt.close()
print(f"Saved: {lr_png}")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
CSV_PATH = r"G:\498R\gridsearch_results.csv"
OUT_DIR  = os.path.dirname(CSV_PATH)

# -----------------------------
# Load results
# -----------------------------
df = pd.read_csv(CSV_PATH)

# Desired order (only keep levels that actually exist in the CSV)
OPTIMIZERS = [o for o in ['Adam','RMSprop','SGD'] if o in set(df['optimizer'])]
SCHEDULERS = [s for s in ['Cosine','Step','Plateau'] if s in set(df['scheduler'])]
BATCH_SIZES = [b for b in [16,32,64] if b in set(df['batch_size'])]
LRS = [lr for lr in [1e-2, 2e-3, 1e-4, 4e-4, 1e-3] if lr in set(df['lr'])]

# -----------------------------
# Colors (same as before)
# -----------------------------
PALETTE_OPT   = ('#D7A4A9', '#A86380')  # rose + plum
PALETTE_SCHED = ('#9CC8B0', '#5E9C8B')  # mint + teal
PALETTE_BS    = ('#BBB3E3', '#7C82B6')  # lavender + slate
LINE_F1_COLOR  = '#E67E22'              # orange
LINE_ACC_COLOR = '#2E8B57'              # sea green

# -----------------------------
# Helpers (transparent, no grid, no value text)
# -----------------------------
def grouped_bar_transparent(filename, labels, vals1, vals2, legend1, legend2, title, ylabel, colors):
    fig, ax = plt.subplots(figsize=(8,6))
    fig.patch.set_alpha(0)           # transparent figure
    ax.set_facecolor('none')         # transparent axes
    x = np.arange(len(labels)); w = 0.36
    ax.bar(x - w/2, vals1, width=w, label=legend1, color=colors[0])
    ax.bar(x + w/2, vals2, width=w, label=legend2, color=colors[1])
    ax.set_xticks(x, labels)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()
    # no grid; small headroom so bars don't touch frame
    ymin, ymax = ax.get_ylim()
    rng = (ymax - ymin) if ymax > ymin else 1.0
    ax.set_ylim(ymin, ymax + 0.05*rng)
    plt.tight_layout()
    out_path = os.path.join(OUT_DIR, filename)
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0.25, transparent=True)
    plt.close()
    print(f"Saved: {out_path}")

def lr_line_transparent(filename, x_vals, y_f1, y_acc, title):
    fig, ax = plt.subplots(figsize=(8,6))
    fig.patch.set_alpha(0)
    ax.set_facecolor('none')
    ax.plot(x_vals, y_f1, marker='o', linewidth=2.5, markersize=8, label='Val Macro-F1', color=LINE_F1_COLOR)
    ax.plot(x_vals, y_acc, marker='o', linewidth=2.5, markersize=8, label='Val Acc', color=LINE_ACC_COLOR)
    ax.set_xscale('log')
    ax.set_xlabel('Learning Rate (log scale)')
    ax.set_ylabel('Score')
    ax.set_title(title)
    ax.legend()
    # small headroom; no grid
    ymin, ymax = ax.get_ylim()
    rng = (ymax - ymin) if ymax > ymin else 1.0
    ax.set_ylim(ymin, ymax + 0.06*rng)
    plt.tight_layout()
    out_path = os.path.join(OUT_DIR, filename)
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0.25, transparent=True)
    plt.close()
    print(f"Saved: {out_path}")

# -----------------------------
# Aggregations (means across other params)
# -----------------------------
agg_opt = df.groupby('optimizer')[['val_macro_f1','val_accuracy']].mean().reindex(OPTIMIZERS)
grouped_bar_transparent(
    filename="hp_optimizer_bar.png",
    labels=agg_opt.index.tolist(),
    vals1=agg_opt['val_macro_f1'].values,
    vals2=agg_opt['val_accuracy'].values,
    legend1='Val Macro-F1', legend2='Val Acc',
    title='Optimizer Comparison (mean across grid)',
    ylabel='Score',
    colors=PALETTE_OPT
)

agg_sch = df.groupby('scheduler')[['val_macro_f1','val_accuracy']].mean().reindex(SCHEDULERS)
grouped_bar_transparent(
    filename="hp_scheduler_bar.png",
    labels=agg_sch.index.tolist(),
    vals1=agg_sch['val_macro_f1'].values,
    vals2=agg_sch['val_accuracy'].values,
    legend1='Val Macro-F1', legend2='Val Acc',
    title='LR Scheduler Comparison (mean across grid)',
    ylabel='Score',
    colors=PALETTE_SCHED
)

agg_bs = df.groupby('batch_size')[['val_macro_f1','val_accuracy']].mean().reindex(BATCH_SIZES)
grouped_bar_transparent(
    filename="hp_batchsize_bar.png",
    labels=[str(x) for x in agg_bs.index.tolist()],
    vals1=agg_bs['val_macro_f1'].values,
    vals2=agg_bs['val_accuracy'].values,
    legend1='Val Macro-F1', legend2='Val Acc',
    title='Batch Size Comparison (mean across grid)',
    ylabel='Score',
    colors=PALETTE_BS
)

agg_lr = df.groupby('lr')[['val_macro_f1','val_accuracy']].mean()
# keep only the LRs present and in desired order
agg_lr = agg_lr.loc[[lr for lr in LRS if lr in agg_lr.index]]
lr_line_transparent(
    filename="hp_lr_sensitivity.png",
    x_vals=np.array(agg_lr.index.values, dtype=float),
    y_f1=agg_lr['val_macro_f1'].values,
    y_acc=agg_lr['val_accuracy'].values,
    title='Learning Rate Sensitivity (mean across grid)'
)